In [1]:
import pandas as pd

In [2]:
# Read the log file line by line and split each line
data = []
with open('anonymized-curriculum-access.txt', 'r') as file:
    for line in enumerate(file):
        print(i)
        parts = line.strip().split()
        timestamp = ' '.join(parts[:2])
        lesson = parts[2]
        user_id = parts[3]
        cohort_id = parts[4]
        ip_address = parts[5]
        data.append((timestamp, lesson, user_id, cohort_id, ip_address))

# Create a DataFrame
columns = ['timestamp', 'lesson', 'user_id', 'cohort_id', 'ip_address']
df = pd.DataFrame(data, columns=columns)

# Display the DataFrame
print(df)

IndexError: list index out of range

In [9]:
with open('anonymized-curriculum-access.txt', 'r') as file:
    for i, line in enumerate(file):
        parts = line.strip().split()
        timestamp = ' '.join(parts[:2])
        print(parts[5])
        
        if i > 3: 
            break

97.105.19.61
97.105.19.61
97.105.19.61
97.105.19.61
97.105.19.61
